# 신경망 모델 구성
신경망의 경우 데이터에 대한 연산을 수행하는 layer, module로 구성.<br>
torch.nn 의 경우 신경망을 구성하는 데 필요한 모든 요소를 제공.
- PyTorch의 모든 모듈은 nn.Module의 하위 클래스, 신경망은 다른 모듈로 구성된 모듈.
    - 이런 중첩구조의 경우 복잡한 아키텍쳐를 쉽게 구축, 관리 가능

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## 클래스 정의
- 신경망 모델을 nn.Module의 하위클래스로 정의. __ init __에서 신경망 계층 초기화.
- nn.Module을 상속받은 forward 메소드에 입력데이터에 대한 연산을 구현

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # 부모클래스 nn.Module의 초기화 매서드 호출
        self.flatten = nn.Flatten() # 이미지 데이터 평탄화하는 nn.Flatten호출 self.flatten에 할당.

        # 신경망 주요부분 정의
        self.linear_relu_stack = nn.Sequential( # nn.Sequential을 사용, 여러 layer를 쌓음 
            nn.Linear(28*28, 512), # 입력크기 28*28(이미지 크기), 출력크기 512인 선형 layer
            nn.ReLU(), # 활성화 함수 정의
            nn.Linear(512, 512), # 입력크기 512, 출력크기 512인 layer
            nn.ReLU(),
            nn.Linear(512, 10), # 입력크기 512, 출력크기 10인 layer
                                # 이때 10은 출력 클래스의 수
        )

    def forward(self, x): # 순전파 정의 메서드
        x = self.flatten(x) # 입력데이터 x를 평탄화
        logits = self.linear_relu_stack(x) # 평탄화 된 데이터를 앞의 신경망 스택에 전달, 출력값(logits)을 얻음
        return logits # 출력값 반환
    

# 정의 된 신경망은 이미지를 입력받아 layer, 활성화 함수 통과 후 최종 10개 클래스에 대한 logit 출력

NeuralNetwork의 인스턴스 생성, 이를 device로 이동후 구조를 출력

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델에 입력을 전달하여 호출 시 2차원 tensor 반환.
- 2차원 tensor dim = 0 : 각 분류(class)에 대한 raw 예측값 10개
- 2차원 tensor dim = 1 : 각 출력의 개별 값이 해당 <br>

raw예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻음

In [5]:
X = torch.rand(1, 28, 28, device=device) # 1개의 샘플을 생성, 각 샘플은 28*28크기 랜덤 이미지
logits = model(X)   # 생성된 이미지를 모델에 전달, logit을 얻음
pred_probab = nn.Softmax(dim=1)(logits) # softmax함수에 logit 전달, 각 클래스에 대한 확률을 얻음
y_pred = pred_probab.argmax(1) # 각 이미지에 대해 예측 확률이 가장 높은 클래스의 index를 찾음(가장 높은확률)
print(f"Predicted class: {y_pred}") # 예측된 클래스 출력, 모델이 랜덤 이미지에 대해 예측한 클래스

Predicted class: tensor([7])


---

## 모델 layer
FashionMNIST 모델의 계층확인.<br>
28*28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망 통과과정 확인

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten 계층을 초기화. 2D 이미지를 784픽셀 값을 갖는 연속된 배열로 반환.

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear 계층은 저장된 가중치와 편향을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈
- 가중치 : 입력 feature와 연결된 파라미터.
- 편향 : 모델이 학습하는 추가적인 파라미터, 입/출력 간의 선형 변환을 조정<br>

선형계층의 출력 = (입력 * 가중치) + 편향

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU : 활성화 함수로 입/출력 간의 복잡한 관계를 생성. 모델 생성 시 다른 활성화 함수 도입가능

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3433, -0.3157, -0.0495, -0.0885,  0.1537, -0.1933, -0.3234, -0.1707,
         -0.1659, -0.0756, -0.0552, -0.0076,  0.1607,  0.0728, -0.3461,  0.4634,
          0.0039, -0.6032, -0.4363,  0.3754],
        [ 0.5423, -0.4544, -0.1555, -0.0017, -0.0186, -0.2037, -0.3845, -0.2941,
          0.1758,  0.2479, -0.2837, -0.1823, -0.1164,  0.3084, -0.2518,  0.1805,
          0.1998, -0.3759, -0.5684,  0.2562],
        [ 0.3560, -0.4805, -0.1927,  0.0030, -0.0044, -0.0904, -0.7096, -0.2607,
         -0.1142, -0.1857,  0.1080, -0.2069, -0.1322,  0.1120, -0.0920,  0.6094,
          0.3018,  0.1943, -0.1354,  0.2115]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3433, 0.0000, 0.0000, 0.0000, 0.1537, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1607, 0.0728, 0.0000, 0.4634, 0.0039, 0.0000,
         0.0000, 0.3754],
        [0.5423, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1758,
         0.2479, 0.0000, 0.0000, 0.0000, 0.3084, 0.00

nn.Sequential은 순서를 갖는 모듈의 컨테이너. sequential container를 사용하여 아래 seq_modules와 같은 신경망을 빠르게 생성 가능

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax : 신경망의 마지막 선형계층은 nn.Softmax 모듈에 전달될 logits를 반환.<br>
logits는 모델의 각 분류에 대한 예측 확률을 나타내도록 0~1 범위로 scale됨
- 즉 특정 클래스에 해당 할 확률을 0~1사이 값으로 반환

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)